In [172]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
#from keras.models import Sequential, Model, load_model, model_from_json
#from keras.layers import Dense
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

"""This neural network is not working well """

'This neural network is not working well '

In [173]:
df = pd.read_csv('/Users/eovito@ibm.com/Desktop/Content/Masterclass/CODE_BASE/bank-additional-full.csv',\
                 sep = ';')

df = df.drop(['duration', 'poutcome'], axis=1)

In [174]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
X, y = df.loc[:, df.columns != 'y'], df['y'].map(dict(yes=1, no=0))
num_cols = [col for col in X_train.columns if np.issubdtype(X_train.dtypes[col], np.number)]
cat_cols = [col for col in X_train.columns if col not in num_cols]

print(num_cols, '\n',  cat_cols)

['age', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'] 
 ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week']


In [175]:
enc = OrdinalEncoder()
enc.fit(X[cat_cols])
X_en = enc.transform(X[cat_cols])
X_en = pd.DataFrame(X_en, columns = X[cat_cols].columns)

X = pd.concat([X_en, X[num_cols]], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
#lets group different types



In [176]:
Z = np.vstack((X_train, X_test))
Zmax = np.max(Z, axis=0)
Zmin = np.min(Z, axis=0)

#normalize an array of samples to range [-0.5, 0.5]
def normalize(V):
    VN = (V - Zmin)/(Zmax - Zmin)
    VN = VN - 0.5
    return(VN)
    
# rescale a sample to recover original values for normalized values. 
def rescale(X):
    return(np.multiply ( X + 0.5, (Zmax - Zmin) ) + Zmin)

N = normalize(Z)
xn_train = N[0:X_train.shape[0], :]
xn_test  = N[X_train.shape[0]:, :]
print(len(xn_train), len(xn_test))

32950 8238


In [186]:
def nn_small():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(10, input_dim=18, kernel_initializer='normal', activation='tanh'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(10, activation='tanh'))
    model.add(keras.layers.Dropout(.2))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(10, activation='tanh'))
    model.add(keras.layers.Dropout(.2))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(10, activation='tanh'))
    model.add(keras.layers.Dropout(.2))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(2, activation= 'softmax'))
    return model

In [213]:
np.random.seed(1) 
tf.set_random_seed(2) 

class_names = ['Not Open', ' Open']

input_shape = ( 1, 18)

# loss function

opt = keras.optimizers.SGD(lr=0.01, decay = 0.001)

def fn(correct, predicted):
    return tf.nn.softmax_cross_entropy_with_logits(labels=correct, logits=predicted)


# compile and print model summary
nn = nn_small()
nn.compile(loss = fn, optimizer = opt, metrics=['accuracy'])
nn.build(input_shape)
nn.summary()


# train model or load a trained model
#if running for the first time change this to True
TRAIN_MODEL = True

if (TRAIN_MODEL):             
    nn.fit(xn_train, y_train, batch_size=64, epochs=75, verbose=1, shuffle=False)
    nn.save_weights(r"/Users/eovito@ibm.com/Desktop/Elyra_Conversion/Elyra\ Build/portuguese_bank_nnsmall.h5")     
else:    
    nn.load_weights("/Users/eovito@ibm.com/Desktop/AIXWorkshops/portugues_bank_nnsmall.h5")
        
# evaluate model accuracy        
score = nn.evaluate(xn_train, y_train, verbose=0) #Compute training set accuracy
#print('Train loss:', score[0])
print('Train accuracy:', score[1])

score = nn.evaluate(xn_test, y_test, verbose=0) #Compute test set accuracy
#print('Test loss:', score[0])
print('Test accuracy:', score[1])

ValueError: Error converting shape to a TensorShape: Dimension -1 must be >= 0.

In [124]:
nn.predict_proba(xn_test[420].reshape((1,) + xn_test[420].shape))

array([[0.5000009, 0.4999991]], dtype=float32)